# template から HTML と JS を生成する


In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

/mnt/e/impc-genes_with_similar_phenotypes/notebooks/notebools-web
/mnt/e/impc-genes_with_similar_phenotypes


# 古いファイルの消去


In [2]:
%%bash

rm web/network/data/*
rm web/network/html/*.html
rm web/network/js/*.js

# index.html に表現型を代入


In [3]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    # print(path_mp_term.stem)
    if not Path("data", "network", f"{mp_term}.json").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    # print(mp_term_name_space)
    mp_terms[mp_term_name_space] = mp_term

In [4]:
print(list(mp_terms)[:3])
print(len(mp_terms))

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
535


In [5]:
import json
json.dump(mp_terms, open("data/available_mp_terms.json", "w"), indent=2)

In [6]:
%%bash

cp data/available_mp_terms.json web/network/data/available_mp_terms.json
grep -v "REMOVE_THIS_LINE" web/network/template/template_index.html > web/index.html

# JSON のコピー


In [7]:
%%bash

cp -r data/network/* web/network/data

cp data/marker_symbol_accession_id.json web/network/data/marker_symbol_accession_id.json

# Template から HTML と JS を生成


In [ ]:
%%bash

########################################
# Extract the HTML and JS files
########################################

find data/mp_term_name -type f |
sed "s|data/mp_term_name/||" |
sed "s|.csv$||" |
while read mp_term_name_underscore; do
    mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
    impc_url=$(grep "$mp_term_name_space" data/mp_term_name_id_url.tsv | cut -f 2)
    # HTML
    cat web/network/template/template.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
    sed "s|XXX_impc_url|${impc_url}|g" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > web/network/html/"$mp_term_name_underscore".html

    # Javascript
    cat web/network/template/template.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > web/network/js/"$mp_term_name_underscore".js
done
